In [1]:
import numpy as np
import pandas as pd
import seaborn as sn

In [2]:
import plotly.graph_objects as go
import urllib, json

In [3]:
import os
from pathlib import Path
from glob import glob
from tqdm import tqdm

In [4]:
#https://www.kaggle.com/competitions/otto-recommender-system/data?select=train.jsonl
path = "C:/Users/AdamS/Downloads/train.jsonl"
data_path = path

In [5]:
#from datasets import load_dataset

In [6]:
#dataset = load_dataset("json", data_files= str(path))

In [7]:
#num_lines = sum(1 for line in open(path))
#print(f'Num. of samples in train dataset: {num_lines:,}')

In [8]:
chunksize = 10000
chunks = pd.read_json(path, lines=True, chunksize=chunksize, orient='index')

In [9]:
num_lines = sum(1 for line in open(data_path))
print(f'number of lines in train: {num_lines:_}')

chunksize = 100_000
num_chunks = int(np.ceil(num_lines / 100_000))
print(f'number of chunks: {num_chunks:_}')

number of lines in train: 12_899_779
number of chunks: 129


In [ ]:
train_sessions = pd.DataFrame()
chunks = pd.read_json(data_path, lines=True, chunksize=100_000)

for e, chunk in enumerate(chunks):
    event_dict = {
        'session': [],
        'aid': [],
        'ts': [],
        'type': [],
    }
    if e < 2:
        # train_sessions = pd.concat([train_sessions, chunk])
        for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
            for event in events:
                event_dict['session'].append(session)
                event_dict['aid'].append(event['aid'])
                event_dict['ts'].append(event['ts'])
                event_dict['type'].append(event['type'])
        chunk_session = pd.DataFrame(event_dict)
        train_sessions = pd.concat([train_sessions, chunk_session])
    else:
        break
        
train_sessions = train_sessions.reset_index(drop=True)

In [ ]:
train_sessions.head()

In [ ]:
train_sessions.groupby('session').head()

In [ ]:
train_sessions.aid.unique().size

In [ ]:
train_sessions.aid.size

In [ ]:
train_sessions.type.unique()

In [ ]:
orders = train_sessions[train_sessions['type'] == ('orders')]

In [ ]:
orders_drop = orders.drop(columns=['session', 'ts', 'type'])

In [ ]:
orders_counts2 = orders_drop.value_counts(ascending = False, normalize = False)

In [ ]:
orders_counts2

In [ ]:
orders3 = orders_counts2.to_frame()
orders3.columns = ['orderCounts']
orders3.head()

In [ ]:
clicks = train_sessions[train_sessions['type'] == ('clicks')]

In [ ]:
clicks_drop = clicks.drop(columns=['session', 'ts', 'type'])

In [ ]:
clicks_counts = clicks_drop.value_counts(ascending = False)

In [ ]:
clicks_counts.head()

In [ ]:
clicks3 = clicks_counts.to_frame()
clicks3.columns = ['clickCounts']
clicks3.head()

In [ ]:
carts = train_sessions[train_sessions['type'] == ('carts')]

In [ ]:
carts_counts = carts.groupby('aid').value_counts(ascending = False)

In [ ]:
carts_drop = carts.drop(columns=['session', 'ts', 'type'])

In [ ]:
carts_counts = carts_drop.value_counts(ascending = False)

In [ ]:
carts3 = carts_counts.to_frame()
carts3.columns = ['cartCounts']
carts3.head()

In [ ]:
countMerge = []

In [ ]:
countMerge = clicks3.join(carts3, how='outer')

In [ ]:
countMerge = countMerge.join(orders3, how='outer')

In [ ]:
countMerge.head()

In [ ]:
countMerge.sort_values('clickCounts', ascending = False).head()

In [ ]:
countMerge.sort_values('orderCounts', ascending = False).head()

In [ ]:
countMerge.sort_values('orderCounts', ascending = False).head()

In [ ]:
countMerge = countMerge.fillna(0)

In [ ]:
countMerge.describe()

In [ ]:
countMerge.index.unique().size

In [ ]:
countMerge.sum()

In [ ]:

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ['Clicked','Carted','Ordered', 'Not Purchased'],
      color = "green"
    ),
    link = dict(
      source = [0,0,1,1], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [1,3,2,3],
      value = [countMerge.cartCounts.sum(), countMerge.clickCounts.sum() - countMerge.cartCounts.sum(), countMerge.orderCounts.sum(), countMerge.cartCounts.sum() - countMerge.orderCounts.sum()]
  ))])

fig.update_layout(title_text="Progression from Click to Order", font_size=10)

fig.show()

In [ ]:
df = train_sessions
df.head()

In [ ]:
df2 = df
df2['clickCount'] = df2.type
df2['cartCount'] = df2.type
df2['orderCount'] = df2.type
df2['clickCount'].values[:] = 0
df2['cartCount'].values[:] = 0
df2['orderCount'].values[:] = 0
df2.loc[df['type'] == 'clicks', 'clickCount'] = 1
df2.loc[df['type'] == 'carts', 'cartCount'] = 1
df2.loc[df['type'] == 'orders', 'orderCount'] = 1
df2.sort_values('type', ascending = False).head(100)